# Clone and install specified Python module

WARNING: We assume that `module` name is the same as the `repo` name.

In [3]:
import os
import subprocess
import importlib
    
CHECK = "\u2705"
FAIL  = "\u274C"
WARN  = "\u26A0"

if IN_COLAB: 
    repo     = GITHUB_REPO
    user     = GITHUB_USER
    folders  = GITHUB_FOLDERS
    
    homedir  = os.getcwd()
    repodir  = f'{homedir}/{repo}'
    
    print('BEGIN{clone and install}\n')
    url      = f'https://github.com/{user}/{repo}'
    raw_url  = url.replace('github.com', 'raw.githubusercontent.com')
    raw_url += '/refs/heads/main/'
    toml_file= 'pyproject.toml'

    # 0. uninstall module
    !pip uninstall -y {repo} 2>/dev/null
    
    # 1. delete local repo folder if it exists
    print(f' 1. delete old repo: {repo}')
    !rm -rf {repo}

    # 2. clone repo
    print(f' 2. clone {repo}')
    !git clone --depth 1 --filter=blob:none --sparse {url}

    print(f' 3. cd {repo}')
    os.chdir(repodir) # move to repo folder
    for folder in folders:
        print(f' 4. check out {folder}')
        !git sparse-checkout set {folder}

    print(f' 5. wget {toml_file}')
    !cd {repodir}
    URL = raw_url + toml_file
    !wget -q {URL}  -O {toml_file}

    print(f' 6. pip install -e .\n')
    !cd {repodir}
    !pip install -e .
    try:
        mod = importlib.import_module(repo)
        print(f"{CHECK} Successfully imported {repo}")
    except ModuleNotFoundError as e:
        print(f"{FAIL} Failed to import '{repo}': {e}")

    print('\nEND{clone and install}')

    !cd {homedir}    

Running locally
